# Training a classifier
We will train a classier for the dataset CIFAR10 that contains images belonging to 10 different classes in two ways:
- training a CNN from scratch
- finetuning a CNN

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
# define transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# define batch size
batch_size = 4

# load train ds
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
# load test ds
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# show some images
import matplotlib.pyplot as plt

def denorm(x):
    out = (x + 1) / 2
    return out.clamp_(0, 1)

to_pil = transforms.ToPILImage()

img, labels = next(iter(trainloader))

plt.imshow(to_pil(torchvision.utils.make_grid(denorm(img))))
plt.axis("off")
plt.show()

# print labels
print(' '.join(f'{classes[labels[j]]}' for j in range(batch_size)))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Define a CNN to classify the images
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # first convolutional block
        self.conv_block1 = nn.Sequential(nn.Conv2d(3, 6, kernel_size=5),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2, 2))
        # second convolutional block
        self.conv_block2 = nn.Sequential(nn.Conv2d(6, 16, kernel_size=5),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2, 2))
        # fully connected blocks
        self.fc1 = nn.Sequential(nn.Linear(16 * 5 * 5, 120),
                                nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(120, 84),
                                 nn.ReLU())
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


net = Net().to(device)
print(net)

In [ ]:
# define Loss and Optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# TRAIN!
# put net into train mode
net.train()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # put data on correct device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
# if you want to save the model
PATH = './res/cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
# if you want to load the model
net.load_state_dict(torch.load(PATH))

# Evaluate the model
## Evaluation Mode (<code>nn.Module.eval()</code>)
Evaluation mode is not actually a mechanism to locally disable gradient computation, but it is sometimes confused to be such a mechanism.

Functionally, <code>module.eval()</code> (or equivalently <code>module.train(False)</code>) are completely orthogonal to no-grad mode and inference mode. How <code>model.eval()</code> affects your model depends entirely on the specific modules used in your model and whether they define any training-mode specific behavior.

You are responsible for calling <code>model.eval()</code> and <code>model.train()</code> if your model relies on modules such as <code>torch.nn.Dropout</code> and <code>torch.nn.BatchNorm2d</code> that may behave differently depending on training mode, for example, to avoid updating your BatchNorm running statistics on validation data.

It is recommended that you always use <code>model.train()</code> when training and <code>model.eval()</code> when evaluating your model (validation/testing) even if you aren’t sure your model has training-mode specific behavior, because a module you are using might be updated to behave differently in training and eval modes.

## No-grad Mode
Computations in no-grad mode behave as if none of the inputs require grad. In other words, computations in no-grad mode are never recorded in the backward graph even if there are inputs that have <code>require_grad=True</code>.

Enable no-grad mode when you need to perform operations that should not be recorded by autograd, but you’d still like to use the outputs of these computations in grad mode later. This context manager makes it convenient to disable gradients for a block of code or function without having to temporarily set tensors to have <code>requires_grad=False</code>, and then back to <code>True</code>.

In [ ]:
# now lets evaluate the model on the test set
correct = 0
total = 0

# put net into evaluation mode
net.eval()

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        # put data on correct device
        inputs, labels = inputs.to(device), labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# Ex1: try to get the accuracy for each class

The classifier was able to correctly classify CIFAR10 with a good accuracy, but we can do much better!
HOW?